# vLLM을 활용한 LLM Agent Serving 튜토리얼

### 에이전트 아키텍처

본 튜토리얼에서 구현할 에이전트는 다음 요소들로 구성된다:
- **Pydantic**: 강력한 타입 검증 및 스키마 정의
- **Function Calling & Tool**: OpenAI 스타일 function calling 지원
- **Memory**: 효율적인 대화 컨텍스트 관리
- **Validation**: 입출력 데이터 무결성 보장
- **Recovery**: 장애 복구 및 재시도 로직
- **Feedback**: 성능 모니터링 및 개선

## 1. 환경 설정 및 라이브러리 설치

In [1]:
%pip install -q asyncio aiohttp

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import asyncio
from typing import List, Dict, Any, Optional, Callable, Union
from datetime import datetime
from pydantic import BaseModel, Field, validator
from openai import OpenAI
import time

## 2. Pydantic 스키마 정의

In [3]:
class FunctionParameter(BaseModel):
    """Function calling의 파라미터를 정의하는 모델이다"""
    type: str = Field(description="파라미터 타입")
    description: str = Field(description="파라미터 설명")
    enum: Optional[List[str]] = Field(default=None, description="가능한 값들의 목록")

class FunctionDefinition(BaseModel):
    """OpenAI 스타일 function 정의를 표현하는 모델이다"""
    name: str = Field(description="함수 이름")
    description: str = Field(description="함수 기능 설명")
    parameters: Dict[str, Any] = Field(description="함수 파라미터 스키마")
    
class ToolDefinition(BaseModel):
    """에이전트가 사용할 도구를 정의하는 모델이다"""
    type: str = Field(default="function", description="도구 타입")
    function: FunctionDefinition = Field(description="함수 정의")
    callable: Callable = Field(description="실제 실행할 함수")
    
    class Config:
        arbitrary_types_allowed = True

class Message(BaseModel):
    """대화 메시지를 표현하는 모델이다"""
    role: str = Field(description="메시지 역할: system, user, assistant, tool")
    content: Optional[str] = Field(default=None, description="메시지 내용")
    tool_calls: Optional[List[Dict[str, Any]]] = Field(default=None, description="도구 호출 정보")
    tool_call_id: Optional[str] = Field(default=None, description="도구 호출 ID")
    name: Optional[str] = Field(default=None, description="도구 이름")
    timestamp: datetime = Field(default_factory=datetime.now, description="메시지 생성 시간")
    
    @validator('role')
    def validate_role(cls, v):
        """역할이 유효한 값인지 검증한다"""
        allowed_roles = ['system', 'user', 'assistant', 'tool']
        assert v in allowed_roles, f"역할은 {allowed_roles} 중 하나여야 한다"
        return v

class AgentResponse(BaseModel):
    """에이전트의 응답을 표현하는 모델이다"""
    content: str = Field(description="응답 내용")
    tool_calls: Optional[List[Dict[str, Any]]] = Field(default=None, description="실행된 도구 정보")
    total_tokens: int = Field(default=0, description="사용된 총 토큰 수")
    latency_ms: float = Field(default=0.0, description="응답 지연 시간")
    is_successful: bool = Field(default=True, description="응답 성공 여부")
    error_message: Optional[str] = Field(default=None, description="오류 메시지")

class ValidationResult(BaseModel):
    """검증 결과를 표현하는 모델이다"""
    is_valid: bool = Field(description="검증 통과 여부")
    error_message: Optional[str] = Field(default=None, description="검증 실패 메시지")
    validation_details: Dict[str, Any] = Field(default_factory=dict, description="상세 검증 정보")

/tmp/ipykernel_321893/3717748012.py:13: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class ToolDefinition(BaseModel):
/tmp/ipykernel_321893/3717748012.py:31: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('role')


## 3. Memory 시스템 구현

In [4]:
class ConversationMemory:
    """대화 기록을 관리하고 컨텍스트 윈도우를 최적화하는 메모리 시스템이다"""
    
    def __init__(self, max_tokens: int = 4096, max_messages: int = 100):
        """최대 토큰 수와 메시지 수를 설정하여 초기화한다"""
        self.messages: List[Message] = []
        self.max_tokens = max_tokens
        self.max_messages = max_messages
        self.system_message: Optional[Message] = None
    
    def set_system_message(self, content: str) -> None:
        """시스템 메시지를 설정한다"""
        self.system_message = Message(role="system", content=content)
    
    def add_message(self, role: str, content: Optional[str] = None, 
                   tool_calls: Optional[List[Dict]] = None,
                   tool_call_id: Optional[str] = None,
                   name: Optional[str] = None) -> None:
        """새로운 메시지를 메모리에 추가한다"""
        message = Message(
            role=role,
            content=content,
            tool_calls=tool_calls,
            tool_call_id=tool_call_id,
            name=name
        )
        self.messages.append(message)
        self._trim_messages()
    
    def _trim_messages(self) -> None:
        """메시지 수가 한계를 초과하면 오래된 메시지를 제거한다"""
        while len(self.messages) > self.max_messages:
            self.messages.pop(0)
    
    def get_messages(self) -> List[Dict[str, Any]]:
        """OpenAI API 형식으로 메시지 리스트를 반환한다"""
        result = []
        
        if self.system_message:
            result.append({"role": "system", "content": self.system_message.content})
        
        for msg in self.messages:
            msg_dict = {"role": msg.role}
            
            if msg.content:
                msg_dict["content"] = msg.content
            if msg.tool_calls:
                msg_dict["tool_calls"] = msg.tool_calls
            if msg.tool_call_id:
                msg_dict["tool_call_id"] = msg.tool_call_id
            if msg.name:
                msg_dict["name"] = msg.name
            
            result.append(msg_dict)
        
        return result
    
    def clear(self) -> None:
        """대화 기록을 초기화한다"""
        self.messages.clear()
    
    def get_statistics(self) -> Dict[str, Any]:
        """메모리 통계 정보를 반환한다"""
        role_counts = {}
        for msg in self.messages:
            role_counts[msg.role] = role_counts.get(msg.role, 0) + 1
        
        return {
            "total_messages": len(self.messages),
            "role_distribution": role_counts,
            "has_system_message": self.system_message is not None
        }

## 4. Function Calling & Tool 시스템 구현

In [5]:
# 예제 도구 함수들을 정의한다

def get_current_time(timezone: str = "UTC") -> str:
    """현재 시간을 반환한다"""
    current_time = datetime.now()
    return f"{timezone} 기준 현재 시간: {current_time.strftime('%Y-%m-%d %H:%M:%S')}다"

def calculate_advanced(expression: str) -> str:
    """수학 표현식을 계산한다"""
    result = eval(expression)
    return f"계산 결과: {expression} = {result}다"

def query_knowledge_base(topic: str, detail_level: str = "medium") -> str:
    """지식 베이스에서 정보를 조회한다"""
    knowledge = {
        "python": "Python은 고수준 프로그래밍 언어다",
        "ai": "인공지능은 기계가 인간의 지능을 모방하는 기술이다",
        "vllm": "vLLM은 고성능 LLM 추론 엔진이다"
    }
    info = knowledge.get(topic.lower(), f"{topic}에 대한 정보를 찾을 수 없다")
    return f"[{detail_level} 수준] {info}"

def execute_database_query(query: str, limit: int = 10) -> str:
    """데이터베이스 쿼리를 실행한다"""
    return f"쿼리 '{query}' 실행 완료: {limit}개의 결과를 반환했다"

In [6]:
class ToolManager:
    """도구들을 관리하고 실행하는 매니저다"""
    
    def __init__(self):
        """도구 매니저를 초기화한다"""
        self.tools: Dict[str, ToolDefinition] = {}
        self.execution_history: List[Dict[str, Any]] = []
    
    def register_tool(self, tool: ToolDefinition) -> None:
        """새로운 도구를 등록한다"""
        self.tools[tool.function.name] = tool
    
    def get_tool_schemas(self) -> List[Dict[str, Any]]:
        """OpenAI API 형식의 도구 스키마를 반환한다"""
        return [
            {
                "type": tool.type,
                "function": {
                    "name": tool.function.name,
                    "description": tool.function.description,
                    "parameters": tool.function.parameters
                }
            }
            for tool in self.tools.values()
        ]
    
    def execute_tool(self, tool_name: str, arguments: Dict[str, Any]) -> str:
        """도구를 실행하고 결과를 반환한다"""
        tool = self.tools.get(tool_name)
        
        start_time = time.time()
        result = tool.callable(**arguments)
        execution_time = (time.time() - start_time) * 1000
        
        # 실행 기록을 저장한다
        self.execution_history.append({
            "tool_name": tool_name,
            "arguments": arguments,
            "result": result,
            "execution_time_ms": execution_time,
            "timestamp": datetime.now()
        })
        
        return result
    
    def get_tool(self, name: str) -> Optional[ToolDefinition]:
        """이름으로 도구를 검색한다"""
        return self.tools.get(name)
    
    def get_execution_statistics(self) -> Dict[str, Any]:
        """도구 실행 통계를 반환한다"""
        if not self.execution_history:
            return {"total_executions": 0}
        
        tool_counts = {}
        total_time = 0
        
        for execution in self.execution_history:
            tool_name = execution["tool_name"]
            tool_counts[tool_name] = tool_counts.get(tool_name, 0) + 1
            total_time += execution["execution_time_ms"]
        
        return {
            "total_executions": len(self.execution_history),
            "tool_usage": tool_counts,
            "average_execution_time_ms": total_time / len(self.execution_history)
        }

## 5. Validation 시스템 구현

In [7]:
class InputValidator:
    """입력 데이터를 검증하는 클래스다"""
    
    @staticmethod
    def validate_message(content: str) -> ValidationResult:
        """메시지 내용의 유효성을 검증한다"""
        details = {}
        
        if not content or not content.strip():
            return ValidationResult(
                is_valid=False,
                error_message="메시지가 비어있다",
                validation_details={"content_length": 0}
            )
        
        content_length = len(content)
        details["content_length"] = content_length
        
        if content_length > 50000:
            return ValidationResult(
                is_valid=False,
                error_message=f"메시지가 너무 길다 (최대 50000자, 현재 {content_length}자)",
                validation_details=details
            )
        
        return ValidationResult(is_valid=True, validation_details=details)
    
    @staticmethod
    def validate_tool_arguments(tool: ToolDefinition, arguments: Dict[str, Any]) -> ValidationResult:
        """도구 호출의 인자를 검증한다"""
        details = {"provided_arguments": list(arguments.keys())}
        
        required_params = tool.function.parameters.get("required", [])
        details["required_parameters"] = required_params
        
        for param in required_params:
            if param not in arguments:
                return ValidationResult(
                    is_valid=False,
                    error_message=f"필수 파라미터 '{param}'가 누락되었다",
                    validation_details=details
                )
        
        return ValidationResult(is_valid=True, validation_details=details)
    
    @staticmethod
    def validate_response(response: Dict[str, Any]) -> ValidationResult:
        """모델 응답의 유효성을 검증한다"""
        details = {}
        
        if "choices" not in response:
            return ValidationResult(
                is_valid=False,
                error_message="응답에 'choices' 필드가 없다",
                validation_details=details
            )
        
        if not response["choices"]:
            return ValidationResult(
                is_valid=False,
                error_message="응답의 choices가 비어있다",
                validation_details=details
            )
        
        details["num_choices"] = len(response["choices"])
        return ValidationResult(is_valid=True, validation_details=details)

## 6. Recovery 시스템 구현

In [8]:
class RecoveryStrategy:
    """오류 발생 시 복구 전략을 관리하는 클래스다"""
    
    def __init__(self, max_retries: int = 3, retry_delay: float = 1.0):
        """복구 전략을 초기화한다"""
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.failure_log: List[Dict[str, Any]] = []
        self.recovery_attempts: Dict[str, int] = {}
    
    def log_failure(self, error_type: str, error_message: str, 
                   context: Dict[str, Any]) -> None:
        """실패 사례를 로그에 기록한다"""
        self.failure_log.append({
            "timestamp": datetime.now(),
            "error_type": error_type,
            "error_message": error_message,
            "context": context
        })
    
    def should_retry(self, operation_id: str) -> bool:
        """재시도 가능 여부를 판단한다"""
        current_attempts = self.recovery_attempts.get(operation_id, 0)
        return current_attempts < self.max_retries
    
    def record_retry(self, operation_id: str) -> int:
        """재시도를 기록하고 현재 시도 횟수를 반환한다"""
        self.recovery_attempts[operation_id] = self.recovery_attempts.get(operation_id, 0) + 1
        return self.recovery_attempts[operation_id]
    
    def reset_operation(self, operation_id: str) -> None:
        """작업의 재시도 카운터를 초기화한다"""
        if operation_id in self.recovery_attempts:
            del self.recovery_attempts[operation_id]
    
    def get_fallback_response(self, error_type: str) -> str:
        """오류 타입에 따른 폴백 응답을 생성한다"""
        fallback_messages = {
            "tool_execution_error": "도구 실행에 실패했다. 다른 방법으로 질문에 답변하겠다.",
            "validation_error": "입력 검증에 실패했다. 올바른 형식으로 다시 질문해 달라.",
            "api_error": "API 호출에 실패했다. 잠시 후 다시 시도해 달라.",
            "timeout_error": "응답 시간이 초과되었다. 더 간단한 질문으로 다시 시도해 달라."
        }
        return fallback_messages.get(error_type, "예상치 못한 오류가 발생했다.")
    
    def get_recovery_statistics(self) -> Dict[str, Any]:
        """복구 통계 정보를 반환한다"""
        error_type_counts = {}
        for failure in self.failure_log:
            error_type = failure["error_type"]
            error_type_counts[error_type] = error_type_counts.get(error_type, 0) + 1
        
        return {
            "total_failures": len(self.failure_log),
            "error_distribution": error_type_counts,
            "active_retries": len(self.recovery_attempts)
        }

## 7. Feedback 시스템 구현

In [9]:
class PerformanceMonitor:
    """에이전트의 성능을 모니터링하고 피드백을 수집하는 클래스다"""
    
    def __init__(self):
        """성능 모니터를 초기화한다"""
        self.response_times: List[float] = []
        self.token_usage: List[int] = []
        self.user_feedback: List[Dict[str, Any]] = []
        self.quality_scores: List[float] = []
    
    def record_response(self, latency_ms: float, tokens: int) -> None:
        """응답 메트릭을 기록한다"""
        self.response_times.append(latency_ms)
        self.token_usage.append(tokens)
    
    def add_user_feedback(self, query: str, response: str, 
                         rating: float, comment: str = "") -> None:
        """사용자 피드백을 추가한다"""
        feedback = {
            "timestamp": datetime.now(),
            "query": query,
            "response": response,
            "rating": rating,
            "comment": comment
        }
        self.user_feedback.append(feedback)
        self.quality_scores.append(rating)
    
    def get_performance_metrics(self) -> Dict[str, Any]:
        """성능 메트릭 요약을 반환한다"""
        metrics = {
            "total_requests": len(self.response_times)
        }
        
        if self.response_times:
            metrics["average_latency_ms"] = sum(self.response_times) / len(self.response_times)
            metrics["min_latency_ms"] = min(self.response_times)
            metrics["max_latency_ms"] = max(self.response_times)
        
        if self.token_usage:
            metrics["average_tokens"] = sum(self.token_usage) / len(self.token_usage)
            metrics["total_tokens"] = sum(self.token_usage)
        
        if self.quality_scores:
            metrics["average_quality_score"] = sum(self.quality_scores) / len(self.quality_scores)
            metrics["total_feedback_count"] = len(self.quality_scores)
        
        return metrics
    
    def generate_insights(self) -> List[str]:
        """수집된 데이터를 기반으로 인사이트를 생성한다"""
        insights = []
        
        if self.response_times:
            avg_latency = sum(self.response_times) / len(self.response_times)
            if avg_latency > 5000:
                insights.append("평균 응답 시간이 5초를 초과한다. 성능 최적화가 필요하다.")
        
        if self.quality_scores:
            avg_quality = sum(self.quality_scores) / len(self.quality_scores)
            if avg_quality < 3.0:
                insights.append("평균 품질 점수가 낮다. 응답 품질 개선이 필요하다.")
            elif avg_quality >= 4.5:
                insights.append("높은 품질 점수를 유지하고 있다.")
        
        if self.token_usage:
            avg_tokens = sum(self.token_usage) / len(self.token_usage)
            if avg_tokens > 3000:
                insights.append("평균 토큰 사용량이 높다. 프롬프트 최적화를 고려해야 한다.")
        
        return insights

## 8. vLLM 에이전트 통합 구현

In [10]:
class VLLMAgent:
    """vLLM을 사용하는 고성능 에이전트 시스템이다"""
    
    def __init__(self, 
                 model_name: str = "openai/gpt-oss-20b",
                 base_url: str = "http://localhost:8001/v1",
                 api_key: str = "EMPTY"):
        """vLLM 에이전트를 초기화하고 모든 컴포넌트를 설정한다"""
        self.model_name = model_name
        self.client = OpenAI(base_url=base_url, api_key=api_key)
        
        # 하위 시스템 초기화
        self.memory = ConversationMemory()
        self.tool_manager = ToolManager()
        self.validator = InputValidator()
        self.recovery_strategy = RecoveryStrategy()
        self.performance_monitor = PerformanceMonitor()
        
        # 시스템 메시지 설정
        self._setup_system_message()
    
    def _setup_system_message(self) -> None:
        """도구 설명을 포함한 시스템 메시지를 설정한다"""
        system_message = """당신은 도구를 사용할 수 있는 유능한 AI 어시스턴트다.
사용자의 질문에 정확하고 도움이 되는 답변을 제공한다.
필요한 경우 제공된 도구를 활용하여 정보를 얻거나 작업을 수행한다."""
        self.memory.set_system_message(system_message)
    
    def _call_vllm(self, messages: List[Dict[str, Any]], 
                  tools: Optional[List[Dict[str, Any]]] = None) -> Dict[str, Any]:
        """vLLM API를 호출하여 응답을 받는다"""
        params = {
            "model": self.model_name,
            "messages": messages,
            "temperature": 0.7,
            "max_tokens": 2000
        }
        
        if tools:
            params["tools"] = tools
            params["tool_choice"] = "auto"
        
        response = self.client.chat.completions.create(**params)
        
        return {
            "choices": [{
                "message": {
                    "role": response.choices[0].message.role,
                    "content": response.choices[0].message.content,
                    "tool_calls": response.choices[0].message.tool_calls
                }
            }],
            "usage": {
                "total_tokens": response.usage.total_tokens
            }
        }
    
    def run(self, user_input: str, max_iterations: int = 10) -> AgentResponse:
        """사용자 입력을 처리하고 응답을 생성한다"""
        start_time = time.time()
        operation_id = f"op_{int(time.time() * 1000)}"
        
        # 입력 검증
        validation = self.validator.validate_message(user_input)
        if not validation.is_valid:
            self.recovery_strategy.log_failure(
                "validation_error", 
                validation.error_message,
                {"input": user_input}
            )
            return AgentResponse(
                content=self.recovery_strategy.get_fallback_response("validation_error"),
                is_successful=False,
                error_message=validation.error_message
            )
        
        # 사용자 메시지 추가
        self.memory.add_message("user", user_input)
        
        tool_calls_made = []
        total_tokens = 0
        
        # 반복적으로 모델 호출 및 도구 실행
        for iteration in range(max_iterations):
            messages = self.memory.get_messages()
            tools = self.tool_manager.get_tool_schemas() if self.tool_manager.tools else None
            
            response = self._call_vllm(messages, tools)
            
            # 응답 검증
            validation = self.validator.validate_response(response)
            if not validation.is_valid:
                if self.recovery_strategy.should_retry(operation_id):
                    self.recovery_strategy.record_retry(operation_id)
                    time.sleep(self.recovery_strategy.retry_delay)
                    continue
                return AgentResponse(
                    content=self.recovery_strategy.get_fallback_response("api_error"),
                    is_successful=False,
                    error_message=validation.error_message
                )
            
            message = response["choices"][0]["message"]
            total_tokens += response["usage"]["total_tokens"]
            
            # 도구 호출이 있는 경우
            if message.get("tool_calls"):
                tool_calls = message["tool_calls"]
                
                # 어시스턴트 메시지 저장
                self.memory.add_message(
                    "assistant",
                    content=message.get("content"),
                    tool_calls=[{
                        "id": tc.id,
                        "type": tc.type,
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments
                        }
                    } for tc in tool_calls]
                )
                
                # 각 도구 호출 실행
                for tool_call in tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    
                    tool = self.tool_manager.get_tool(function_name)
                    
                    # 도구 인자 검증
                    arg_validation = self.validator.validate_tool_arguments(tool, function_args)
                    if not arg_validation.is_valid:
                        tool_result = f"오류: {arg_validation.error_message}"
                    else:
                        tool_result = self.tool_manager.execute_tool(function_name, function_args)
                    
                    tool_calls_made.append({
                        "tool": function_name,
                        "arguments": function_args,
                        "result": tool_result
                    })
                    
                    # 도구 실행 결과를 메모리에 추가
                    self.memory.add_message(
                        "tool",
                        content=tool_result,
                        tool_call_id=tool_call.id,
                        name=function_name
                    )
            else:
                # 일반 응답인 경우 종료
                content = message.get("content", "")
                self.memory.add_message("assistant", content)
                
                latency_ms = (time.time() - start_time) * 1000
                self.performance_monitor.record_response(latency_ms, total_tokens)
                self.recovery_strategy.reset_operation(operation_id)
                
                return AgentResponse(
                    content=content,
                    tool_calls=tool_calls_made if tool_calls_made else None,
                    total_tokens=total_tokens,
                    latency_ms=latency_ms,
                    is_successful=True
                )
        
        # 최대 반복 도달
        latency_ms = (time.time() - start_time) * 1000
        return AgentResponse(
            content="최대 반복 횟수에 도달했다. 현재까지의 결과를 반환한다.",
            tool_calls=tool_calls_made if tool_calls_made else None,
            total_tokens=total_tokens,
            latency_ms=latency_ms,
            is_successful=True
        )
    
    def add_feedback(self, query: str, response: str, rating: float, comment: str = "") -> None:
        """사용자 피드백을 추가한다"""
        self.performance_monitor.add_user_feedback(query, response, rating, comment)
    
    def get_agent_status(self) -> Dict[str, Any]:
        """에이전트의 전체 상태를 반환한다"""
        return {
            "memory_stats": self.memory.get_statistics(),
            "tool_stats": self.tool_manager.get_execution_statistics(),
            "recovery_stats": self.recovery_strategy.get_recovery_statistics(),
            "performance_metrics": self.performance_monitor.get_performance_metrics(),
            "insights": self.performance_monitor.generate_insights()
        }

## 9. 도구 등록 및 에이전트 초기화

In [11]:
# vLLM 에이전트 초기화
agent = VLLMAgent(
    model_name="openai/gpt-oss-20b",
    base_url="http://localhost:8001/v1"
)

# 시간 조회 도구 등록
time_tool = ToolDefinition(
    type="function",
    function=FunctionDefinition(
        name="get_current_time",
        description="현재 시간을 조회한다",
        parameters={
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "시간대 (예: UTC, KST)",
                    "enum": ["UTC", "KST", "EST", "PST"]
                }
            },
            "required": []
        }
    ),
    callable=get_current_time
)
agent.tool_manager.register_tool(time_tool)

# 계산기 도구 등록
calculator_tool = ToolDefinition(
    type="function",
    function=FunctionDefinition(
        name="calculate_advanced",
        description="복잡한 수학 표현식을 계산한다",
        parameters={
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "계산할 수학 표현식 (예: '2 + 3 * 4')"
                }
            },
            "required": ["expression"]
        }
    ),
    callable=calculate_advanced
)
agent.tool_manager.register_tool(calculator_tool)

# 지식 베이스 조회 도구 등록
knowledge_tool = ToolDefinition(
    type="function",
    function=FunctionDefinition(
        name="query_knowledge_base",
        description="지식 베이스에서 정보를 검색한다",
        parameters={
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "검색할 주제"
                },
                "detail_level": {
                    "type": "string",
                    "description": "상세 수준",
                    "enum": ["low", "medium", "high"]
                }
            },
            "required": ["topic"]
        }
    ),
    callable=query_knowledge_base
)
agent.tool_manager.register_tool(knowledge_tool)

# 데이터베이스 쿼리 도구 등록
db_tool = ToolDefinition(
    type="function",
    function=FunctionDefinition(
        name="execute_database_query",
        description="데이터베이스 쿼리를 실행한다",
        parameters={
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "실행할 SQL 쿼리"
                },
                "limit": {
                    "type": "integer",
                    "description": "결과 개수 제한"
                }
            },
            "required": ["query"]
        }
    ),
    callable=execute_database_query
)
agent.tool_manager.register_tool(db_tool)

print("vLLM 에이전트가 초기화되었다")
print(f"등록된 도구: {list(agent.tool_manager.tools.keys())}")

vLLM 에이전트가 초기화되었다
등록된 도구: ['get_current_time', 'calculate_advanced', 'query_knowledge_base', 'execute_database_query']


## 10. 에이전트 실행 예제

In [12]:
# 예제 1: 일반 대화
print("=== 예제 1: 일반 대화 ===")
response = agent.run("안녕하세요! vLLM에 대해 간단히 설명해주세요.")
print(f"응답: {response.content}")
print(f"토큰 사용량: {response.total_tokens}")
print(f"지연 시간: {response.latency_ms:.2f}ms")
print()

=== 예제 1: 일반 대화 ===
응답: 안녕하세요! vLLM은 **GPU 메모리와 연산 효율을 극대화한 대형 언어 모델(LLM) 추론 프레임워크**입니다.

- **핵심 아이디어**  
  - **동적 배치(Dynamic batching)**: 실시간 요청을 작은 배치로 묶어 GPU 사용률을 높입니다.  
  - **KV 캐시 공유(Cache sharing)**: 같은 프롬프트를 가진 요청은 KV 캐시를 공유해 메모리 사용량을 줄이고 속도를 끌어올립니다.  
  - **삼각형 행렬 분산(Tensor parallelism)**: 모델을 여러 GPU에 나누어 파라미터를 분산시켜 대형 모델도 한 대의 GPU에서 실행 가능하도록 합니다.

- **주요 특징**  
  - **높은 처리량**: 동일한 하드웨어에서도 기존 라이브러리보다 2~3배 빠른 추론 속도 제공.  
  - **저렴한 비용**: 메모리 효율 덕분에 GPU 수를 줄여 비용 절감.  
  - **간단한 API**: Hugging Face Transformers와 호환되는 `LLMEngine` 인터페이스로 바로 사용 가능.  
  - **멀티-모델 지원**: 하나의 GPU에서 여러 모델을 동시에 서비스할 수 있는 “모델 스위칭” 기능 제공.

- **사용 사례**  
  - 대규모 챗봇, 문서 생성, 번역 서비스 등 실시간 반응이 필요한 애플리케이션에 적합합니다.  
  - 연구실이나 기업이 자체 LLM을 빠르게 프로토타이핑하거나 운영 환경에 배포할 때 활용됩니다.

간단히 말해, vLLM은 **GPU 리소스를 최대한 활용해 LLM 추론을 빠르고 비용 효율적으로** 만들어 주는 도구입니다. 필요하면 더 깊이 들어가서 구현 예시나 성능 비교도 알려드릴게요!
토큰 사용량: 875
지연 시간: 2502.49ms



In [13]:
# 예제 2: 도구 사용 - 계산기
print("=== 예제 2: 계산기 도구 사용 ===")
response = agent.run("(15 + 25) * 3 - 10을 계산해주세요")
print(f"응답: {response.content}")
print(f"도구 호출: {response.tool_calls}")
print(f"지연 시간: {response.latency_ms:.2f}ms")
print()

=== 예제 2: 계산기 도구 사용 ===
응답: (15 + 25) × 3 – 10 = **110**
도구 호출: None
지연 시간: 414.47ms



In [14]:
# 예제 3: 도구 사용 - 시간 조회
print("=== 예제 3: 시간 조회 도구 사용 ===")
response = agent.run("현재 KST 시간을 알려주세요")
print(f"응답: {response.content}")
print(f"도구 호출: {response.tool_calls}")
print()

=== 예제 3: 시간 조회 도구 사용 ===
응답: {"timezone":"KST"}
도구 호출: None



In [15]:
# 예제 4: 복합 도구 사용
print("=== 예제 4: 복합 도구 사용 ===")
response = agent.run("vLLM에 대한 정보를 검색하고, 그 결과를 요약해주세요")
print(f"응답: {response.content}")
print(f"도구 호출 수: {len(response.tool_calls) if response.tool_calls else 0}")
print()

=== 예제 4: 복합 도구 사용 ===
응답: {"topic":"vLLM","detail_level":"medium"}
도구 호출 수: 0



In [16]:
# 예제 5: 피드백 추가
print("=== 예제 5: 피드백 추가 ===")
query = "Python에 대한 정보를 알려주세요"
response = agent.run(query)
agent.add_feedback(query, response.content, rating=4.5, comment="매우 상세하고 정확한 답변이다")
print(f"피드백이 추가되었다")
print()

=== 예제 5: 피드백 추가 ===
피드백이 추가되었다



## 11. 성능 모니터링 및 분석

In [17]:
print("=== 성능 메트릭 ===")
status = agent.get_agent_status()

print("\n메모리 통계:")
for key, value in status["memory_stats"].items():
    print(f"  {key}: {value}")

print("\n도구 실행 통계:")
for key, value in status["tool_stats"].items():
    print(f"  {key}: {value}")

print("\n성능 메트릭:")
for key, value in status["performance_metrics"].items():
    print(f"  {key}: {value}")

print("\n인사이트:")
for insight in status["insights"]:
    print(f"  - {insight}")

=== 성능 메트릭 ===

메모리 통계:
  total_messages: 10
  role_distribution: {'user': 5, 'assistant': 5}
  has_system_message: True

도구 실행 통계:
  total_executions: 0

성능 메트릭:
  total_requests: 5
  average_latency_ms: 1397.0731735229492
  min_latency_ms: 264.71614837646484
  max_latency_ms: 3445.158004760742
  average_tokens: 1060.4
  total_tokens: 5302
  average_quality_score: 4.5
  total_feedback_count: 1

인사이트:
  - 높은 품질 점수를 유지하고 있다.


## 12. 메모리 관리 테스트

In [18]:
print("=== 메모리 관리 테스트 ===")

# 첫 번째 대화
response1 = agent.run("제 이름은 박지민이고, 데이터 과학자입니다")
print(f"응답 1: {response1.content}")

# 두 번째 대화 - 이전 컨텍스트 참조
response2 = agent.run("제 직업이 무엇이었죠?")
print(f"응답 2: {response2.content}")

# 세 번째 대화 - 복합 질문
response3 = agent.run("제 이름과 직업을 활용해서 간단한 자기소개를 작성해주세요")
print(f"응답 3: {response3.content}")

=== 메모리 관리 테스트 ===
응답 1: 안녕하세요, 박지민님!  
데이터 과학자이시라니, 정말 멋지죠. vLLM, Python, 혹은 머신러닝, 통계, 데이터 시각화 등 어떤 주제든 궁금한 점이 있으면 언제든 물어보세요. 도와드릴게요!
응답 2: 당신의 직업은 **데이터 과학자**입니다.
응답 3: 안녕하세요! 저는 박지민입니다. 데이터 과학자로서, 빅데이터와 머신러닝을 활용해 인사이트를 도출하고 비즈니스 가치를 창출하는 일을 하고 있습니다. 앞으로도 새로운 데이터 도전과 학습을 즐기며 성장해 나가겠습니다.


## 13. Recovery 시스템 테스트

In [19]:
print("=== Recovery 시스템 테스트 ===")

# 빈 입력 테스트
response = agent.run("")
print(f"빈 입력 응답: {response.content}")
print(f"성공 여부: {response.is_successful}")
print(f"오류 메시지: {response.error_message}")

# 복구 통계 확인
recovery_stats = agent.recovery_strategy.get_recovery_statistics()
print(f"\n복구 통계: {recovery_stats}")

=== Recovery 시스템 테스트 ===
빈 입력 응답: 입력 검증에 실패했다. 올바른 형식으로 다시 질문해 달라.
성공 여부: False
오류 메시지: 메시지가 비어있다

복구 통계: {'total_failures': 1, 'error_distribution': {'validation_error': 1}, 'active_retries': 0}
